## Critical path showcase



This notebook provides examples of a simulation that takes a number of barges and a total amount to be moved. We show the critical path (as a pd.DataFrame and a gantt plot) for a number of different simulation (data) and methods.

For this example we work with the following methods to determine critical path:
* __DependenciesFromRecordedActivities__
* __DependenciesFromSimpy__

The latter method is recommended as the dependencies found with `from simpy` correspond wth the events as scheduled within the simulation, whereas `from recorded activities` assumes dependencies from recorded data (dependencies assumed in case of identical timestamps and simulation objects).

For this example we work with the following simulations to determine critical path:
* two barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (third) vessel may move the amount from site 1 to site 2.
* four barges move an amount from site 0 to site 1. When the container on the site 1 reaches a certain level, another (fifth) vessel moves the entire amount from site 1 to site 2.

Note that the barges have a start event (delay) and velocity based on their number (i.e. barge 0 has zero delay, barge1 100seconds, barge2 200 seconds et cetera).


#### 0. Import libraries

In [1]:
import simpy
import shapely

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

from openclsim.critical_path.dependencies_from_simpy_step import DependenciesFromSimpy, CriticalPathEnvironment
from openclsim.critical_path.dependencies_from_recorded_activities import DependenciesFromRecordedActivities


#### 1. Use a function definition to get some simulation data

In this notebook we can play around with the nr of barges and total amount to be moved. After calling this function with desired input parameters we can investigate the data of the simulations and inspect the gantt chart.

In [2]:
def demo_data(nr_barges, total_amount, env=None):
    """
    Run a simulation where <nr_barges> barges need to shift an amount of <total_amount>
    from site 1 (nr resources=1) to site 2 (nr resources=4) whereafter another vessel can come into action.

    Parameters
    ----------
    nr_barges : int
        Number of barges in the simulation.
    total_amount : int
        Total amount to be transported in the simulation.
    env : simpy.Environment or class that inherits from simpy.Environment
        Optional. If None, default to simpy Environment
    """
    Site = type(
        "Site",
        (
            core.Identifiable,
            core.Log,
            core.Locatable,
            core.HasContainer,
            core.HasResource,
        ),
        {},
    )
    TransportProcessingResource = type(
        "TransportProcessingResource",
        (
            core.Identifiable,
            core.Log,
            core.ContainerDependentMovable,
            core.Processor,
            core.HasResource,
        ),
        {},
    )

    simulation_start = 0
    if env is None:
        my_env = simpy.Environment(initial_time=simulation_start)
    else:
        my_env = env(initial_time=simulation_start)

    registry = {}

    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

    data_from_site = {
        "env": my_env,
        "name": "from_site",
        "geometry": location_from_site,
        "capacity": 4 * total_amount,
        "level": 4 * total_amount,
        "nr_resources": 1,
    }
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {
        "env": my_env,
        "name": "to_site",
        "geometry": location_to_site,
        "capacity": 3*total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site = Site(**data_to_site)

    location_to_site2 = shapely.geometry.Point(4.35222222, 52.11428333)
    data_to_site2 = {
        "env": my_env,
        "name": "to_site2",
        "geometry": location_to_site2,
        "capacity": total_amount,
        "level": 0,
        "nr_resources": 4,
    }
    to_site2 = Site(**data_to_site2)

    vessels = {}

    for i in range(nr_barges):
        vessels[f"vessel{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=10,
            compute_v=lambda x: 10,
        )

    # vessel_last wait till whiletask done
    vessel_last = TransportProcessingResource(
        env=my_env,
        name="vessel_last",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10,
    )
    vessels["vessel_last"] = vessel_last

    activities = {}
    for i in range(nr_barges):
        amount = 5  # handle loading
        duration = 2000  # sailing and unloading

        requested_resources = {}
        activities[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle{i}",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name=f"sequential_activity_subcycle{i}",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessels[f"vessel{i}"]],
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=from_site,
                            duration=duration - 200*i
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=from_site,
                            destination=vessels[f"vessel{i}"],
                            amount=amount,
                            duration=500 * amount,
                            requested_resources=requested_resources,
                            start_event=[
                            {
                                "type": "container",
                                "concept": from_site,
                                "state": "gt",
                                "level": amount
                            }
                            ]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            mover=vessels[f"vessel{i}"],
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading:" + vessels[f"vessel{i}"].name,
                            registry=registry,
                            processor=vessels[f"vessel{i}"],
                            origin=vessels[f"vessel{i}"],
                            destination=to_site,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": from_site,
                    "state": "lt",
                    "level": 2.8*total_amount
                }
            ],
              start_event={
                "type":"time",
                "start_time":i*100,
            }
        )

    # now add activity for vessel last, once v1 and v2 are done
    requested_resources = {}
    amount = 5
    duration = 2000
    activities[f"activity_last_vessel"] = model.WhileActivity(
            env=my_env,
            name=f"while_last_vessel",
            registry=registry,
            sub_processes=[
                model.SequentialActivity(
                    env=my_env,
                    name="sequential_last_vessel",
                    registry=registry,
                    sub_processes=[
                        model.BasicActivity(
                            env=my_env,
                            name="basic activity vessel_last",
                            registry=registry,
                            duration=duration,
                            additional_logs=[vessel_last]
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing empty: vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="loading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=to_site,
                            destination=vessel_last,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                        model.MoveActivity(
                            env=my_env,
                            name="sailing full vessel_last",
                            registry=registry,
                            mover=vessel_last,
                            destination=to_site2,
                            duration=duration,
                        ),
                        model.ShiftAmountActivity(
                            env=my_env,
                            name="unloading vessel_last",
                            registry=registry,
                            processor=vessel_last,
                            origin=vessel_last,
                            destination=to_site2,
                            amount=amount,
                            duration=duration,
                            requested_resources=requested_resources,
                        ),
                    ],
                )],
            start_event=[
                {
                    "type": "container",
                     "concept": to_site,
                     "state": "gt",
                     "level": 30
                }
            ],
            condition_event=[
                {
                    "type": "container",
                    "concept": to_site2,
                    "state": "full",
                }
            ],)

    model.register_processes(list(activities.values()))
    my_env.run()

    return {
        "env": my_env,
        "object_list": [from_site, to_site, to_site2] + list(vessels.values()),
        "activity_list": list(activities.values()),
    }



#### 2. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 2 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [3]:
# Run the simulation.
simulation_data_out = demo_data(2, 100, env=CriticalPathEnvironment)

In [4]:
# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,dfc5c99d-f234-43b9-befd-a91b7aca35a1,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,27d0ed0d-b005-4fd0-8d12-245441e471c1,False
9,92b194e3-3178-4961-b8c7-0690311bec12,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,9cb85bc0-64d1-482a-90cd-060db32a0a9d,True
15,d4c8faf3-4a67-4181-bae6-77bdba5e23ef,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,f21b2b49-e6b4-487d-bd3a-493543b6a08c,False
18,bf59c117-3531-45cd-b567-f44e54e60772,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,69599d8b-ad52-4fdc-9127-5b4f35929e3f,True
21,fa6b6c54-7a24-45d8-9b4c-ea1eba5f2c62,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,b1d98585-900c-4945-8f0a-1894cecb0aff,True


In [5]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 3. Get critical path with recorded activities

* _method: DependenciesFromRecordedActivities_
* _simulation: simulation with 2 barges_

First we run the simulation, then we get the recorded activities (with flag 'is_critical') and show these in a gantt plot.

In [6]:
# Run the simulation.
simulation_data_out = demo_data(2, 100)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_activities = DependenciesFromRecordedActivities(**simulation_data_out)
critical_df = my_cp_dependencies_from_activities.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,e2e852e5-8843-48e0-8f7c-94172100259f,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,ff9b7e9f-e279-4e2d-89ae-a77263e48173,False
9,73990b7f-e5ca-4fb5-84ab-e1eec1c8cd3e,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,d0f1808c-1045-41b3-aeb9-ff7338213e5f,False
15,686dcdd1-214f-4e28-9c95-66806da81f47,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,b379ef15-0690-4144-8d06-e91d3420551a,False
18,5308b305-6fe9-4f33-ba6f-6dcaf6b5466b,sailing empty:barge_1,barge_1,1970-01-01 00:35:00,1970-01-01 01:05:00,0 days 00:30:00,ACTIVE,99cfbee6-583a-43fb-b5fe-babdbb8b56e7,False
21,11e43a25-f320-47e2-b3fa-b84fcc5e676e,loading:barge_1,barge_1,1970-01-01 01:05:00,1970-01-01 01:46:40,0 days 00:41:40,ACTIVE,b8fe4b74-d261-4ca1-b46f-a9273753fa06,False


In [7]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()

#### 4. Get critical path with simpy

* _method: DependenciesFromSimpy_
* _simulation: simulation with 4 barges_

First we run the simulation, then we initialize the Dependencies instance in order to get the recorded activities (with flag 'is_critical') and show these in a Gantt plot.

In [8]:
# Run the simulation.
simulation_data_out = demo_data(4, 100, env=CriticalPathEnvironment)

# Initialize the Dependencies instance and get the critical path (as a pd.DataFrame).
my_cp_dependencies_from_simpy = DependenciesFromSimpy(**simulation_data_out)
critical_df = my_cp_dependencies_from_simpy.get_critical_path_df()
critical_df.head()

,ActivityID,Activity,SimulationObject,start_time,end_time,duration,state,cp_activity_id,is_critical
1,ce1b2f4b-a48e-40b6-8504-b4d328903981,basic activity:barge_0,barge_0,1970-01-01 00:00:00,1970-01-01 00:33:20,0 days 00:33:20,ACTIVE,68151a02-188b-4f95-95c2-46149b5c7730,False
11,745e0493-df32-45cd-8b1d-13c4fff0a604,basic activity:barge_1,barge_1,1970-01-01 00:01:40,1970-01-01 00:35:00,0 days 00:33:20,ACTIVE,68716b02-16d5-4432-bc33-5f8fa8c4a51b,False
17,69211222-3a18-440b-ba9b-4821a4cbacdf,basic activity:barge_2,barge_2,1970-01-01 00:03:20,1970-01-01 00:36:40,0 days 00:33:20,ACTIVE,38d14535-d64a-4c1d-bd34-c6e0ee41e4f2,False
23,9b45f76e-1624-46b9-8fef-64ef9ab8b387,basic activity:barge_3,barge_3,1970-01-01 00:05:00,1970-01-01 00:38:20,0 days 00:33:20,ACTIVE,0c4b9607-58de-446c-91e2-20961f2d91fa,True
29,3eeca637-ba18-420d-866c-4627748193c9,sailing empty:barge_0,barge_0,1970-01-01 00:33:20,1970-01-01 01:06:40,0 days 00:33:20,ACTIVE,2a010ee3-54e5-484f-ae98-6abbc133adc5,False


In [9]:
# Show the critical path (within a plotly Gannt plot).
my_cp_dependencies_from_simpy.make_plotly_gantt_chart()